# Отток клиентов

Из банка стали уходить клиенты. Маркетологи посчитали, что сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. В нашем распоряжении исторические данные о поведении клиентов и расторжении договоров с банком. 

**Описание данных:**

**Признаки**
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — наличие недвижимости у клиента
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

**Целевой признак**
- Exited — факт ухода клиента

**Задача** - Построить модель с предельно большим значением *F1*-меры.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

## Подготовка данных

### Загрузка и изучение данных

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
display(data.head(10))
display(data.info())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

Таблица содержит 14 столбцов.

1. Столбцы: "RowNumber", "CustomerId", "Surname" являются идентефикаторами каждого конкретного клиента. Эти данные никак не влияют на целевой признак, а для идентификации клиента достаточно одного столбца "CustomerId" поэтому остальные нужно удалить, чтобы они не оказывали влияния на обучающуюся модель.
2. Целевой признак - "Exited".
3. Остальные столбцы - признаки.

В столбце "Tenure" присутствуют пропуски. Возможно, пропуски означают что у клиента нет недвижимости, поэтому заменим пропуски нулями. После замены пропусков на нули, изменим тип данных на целочисленный

In [4]:
data = data.drop(['RowNumber', 'Surname'], axis=1)

In [5]:
data['Tenure'].fillna(0, inplace=True)
data['Tenure'] = data['Tenure'].astype(int)
print(data['Tenure'].value_counts())

0     1291
1      952
2      950
8      933
3      928
5      927
7      925
4      885
9      882
6      881
10     446
Name: Tenure, dtype: int64


### Кодирование признаков

В данных присутствуют два категориальных признака, заданных текстом. Нам необходимо исследовать различные модели, а логистическая регрессия не работает с категориальными признаками. Поэтому, переведем эти признаки в численные техникой прямого кодирования OHE.

Для начала убедимся, что данные в столбцах с категориальными признакими заданы корректно, без опечаток и каждое значение уникально.

In [6]:
print(data['Geography'].unique())
print('')
print(data['Geography'].value_counts())
print('')
print(data['Gender'].unique())
print('')
print(data['Gender'].value_counts())

['France' 'Spain' 'Germany']

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

['Female' 'Male']

Male      5457
Female    4543
Name: Gender, dtype: int64


Данные заданы корректно, без опечаток, пропусков нет. Закодируем признаки, придав аргументу drop_first значение True, т.к. новые столбцы сильно связаны между собой.

In [7]:
data_ohe = pd.get_dummies(data, drop_first = True)

In [8]:
display(data_ohe.head())

,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,15634602,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,15647311,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,15619304,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,15701354,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,15737888,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


### Разбиение данных на выборки

Разобъем данные на обучающую, тестовую и валидационную выборки.

In [9]:
pre_train, test = train_test_split(data_ohe, test_size=0.2, random_state=123, stratify=data_ohe['Exited'])
train, valid = train_test_split(pre_train, test_size=0.25, random_state=123, stratify=pre_train['Exited'])

print(train.shape)
print(valid.shape)
print(test.shape)

(6000, 13)
(2000, 13)
(2000, 13)


Создадим переменные с признаками и с целевым признаком.

In [10]:
test_features = test.drop(['CustomerId', 'Exited'], axis=1)
test_target = test['Exited']
print(test_features.shape)
print(test_target.shape)
print('')

train_features = train.drop(['CustomerId', 'Exited'], axis=1)
train_target = train['Exited']
print(train_features.shape)
print(train_target.shape)
print('')

valid_features = valid.drop(['CustomerId', 'Exited'], axis=1)
valid_target = valid['Exited']
print(valid_features.shape)
print(valid_target.shape)
print('')

(2000, 11)
(2000,)

(6000, 11)
(6000,)

(2000, 11)
(2000,)



### Масштабирование признаков

В данной задаче будем считать признаким равнозначимыми. Для этого проведем масштабирование численных признаков методом стандартизации данных. Мы кодировали данные техникой OHE, которая переводит категориальные признаки в численные, поэтому все наши признаки сейчас численные. Масштабирование применим ко всем признакам.

In [11]:
print(train_features.columns)

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Geography_Germany',
       'Geography_Spain', 'Gender_Male'],
      dtype='object')


In [12]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Geography_Germany',
       'Geography_Spain', 'Gender_Male']

scaler = StandardScaler()
scaler.fit(train_features)

train_features[numeric] = scaler.transform(train_features[numeric])
test_features[numeric] = scaler.transform(test_features[numeric])
valid_features[numeric] = scaler.transform(valid_features[numeric])

## Исследование задачи

### Дерево решений

Обучим модель решающего дерева и проверим метрику accuracy.

In [13]:
model_tree = DecisionTreeClassifier(random_state=123)
model_tree.fit(train_features, train_target)
tree_valid_predictions = model_tree.predict(valid_features)
print('Accuracy =', accuracy_score(valid_target, tree_valid_predictions))
print('F1 =', f1_score(valid_target, tree_valid_predictions))

Accuracy = 0.8005
F1 = 0.5055762081784386


Построим константную модель, отвечающую всегда 0, проверим ее метрику accuracy, она же будет равна процентному отношению отрицательных ответов.

In [14]:
const_model = DummyClassifier(strategy='constant', constant=0)
const_model.fit(train_features, train_target)
print('Accuracy константной модели =', const_model.score(valid_features, valid_target))

Accuracy константной модели = 0.7965


**Вывод**

Модель дерева решений предсказывает ответы хуже, чем костантная, это никуда не годиться.

Наблюдается достаточно сильный дисбаланс классов, равный 1:5 положительных ответов к отрицательным

Проверим модели логистической регрессии и случайного леса.

### Случайный лес

In [15]:
model_forest = RandomForestClassifier(random_state=123)
model_forest.fit(train_features, train_target)
forest_valid_predictions = model_forest.predict(valid_features)
print('Accuracy =', accuracy_score(valid_target, forest_valid_predictions))
print('F1 =', f1_score(valid_target, forest_valid_predictions))

Accuracy = 0.847
F1 = 0.5377643504531722


Случайный лес предсказывает ответы лучше, чем дерево решений, но accuracy и f1 всё ещё низкие.

### Логистическая регрессия

In [16]:
model_logreg = LogisticRegression(random_state=123)
model_logreg.fit(train_features, train_target)
logreg_valid_predictions = model_logreg.predict(valid_features)
print('Accuracy =', accuracy_score(valid_target, logreg_valid_predictions))
print('F1 =', f1_score(valid_target, logreg_valid_predictions))

Accuracy = 0.804
F1 = 0.3217993079584775


### Вывод

В ответах наблюдается дисбаланс классов. Соотношение положительных и отрицательных ответов 1:5.
Модели, обученные без учета дисбаланса классов не дают достаточных показателей метрики f1.

## Борьба с дисбалансом

Устраним дисбаланс классов разными методами и посмотрим на полученный результат.

### Upsampling

Устраним дисбаланс классов методом upsampling

In [17]:
def upsample(features, target, repeat):
    
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=123)
    
    return features_upsampled, target_upsampled

train_features_upsampled, train_target_upsampled = upsample(train_features, train_target, 5)

print(train_features.shape)
print(train_features_upsampled.shape)
print(train_target_upsampled.shape)

(6000, 11)
(10892, 11)
(10892,)


Обучим разные модели и посчитаем метрики accuracy, f1, auc-roc.

**Дерево решений**

In [18]:
best_accuracy = 0
best_f1 = 0
best_auc_roc = 0
best_depth = 0
best_model_tree_upsampled = None

for depth in range(1,16):      
    model_tree_upsampled = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model_tree_upsampled.fit(train_features_upsampled, train_target_upsampled)
    tree_upsampled_valid_predictions = model_tree_upsampled.predict(valid_features)
    tree_upsampled_prob = model_tree_upsampled.predict_proba(valid_features)
    tree_upsampled_prob_one = tree_upsampled_prob[:, 1]
    
    if f1_score(valid_target, tree_upsampled_valid_predictions) > best_f1:
        best_accuracy = accuracy_score(valid_target, tree_upsampled_valid_predictions)
        best_f1 = f1_score(valid_target, tree_upsampled_valid_predictions)
        best_auc_roc = roc_auc_score(valid_target, tree_upsampled_prob_one)
        best_depth = depth
        best_model_tree_upsampled = model_tree_upsampled
        
print('Best depth =', best_depth)
print('Accuracy =', best_accuracy)
print('F1 =', best_f1)
print('auc_roc =', best_auc_roc)
print('')

Best depth = 7
Accuracy = 0.758
F1 = 0.5686274509803922
auc_roc = 0.8168546049901981



**Случайный лес**

In [19]:
best_accuracy = 0
best_f1 = 0
best_auc_roc = 0
best_depth = 0
best_est = 0
best_model_forest_upsampled = None

for est in range(1, 16):
    for depth in range(1, 16):
        model_forest_upsampled = RandomForestClassifier(n_estimators = est, random_state=123, max_depth=depth)
        model_forest_upsampled.fit(train_features_upsampled, train_target_upsampled)
        forest_upsampled_valid_predictions = model_forest_upsampled.predict(valid_features)
        forest_upsampled_prob = model_forest_upsampled.predict_proba(valid_features)
        forest_upsampled_prob_one = forest_upsampled_prob[:, 1]
    
        if f1_score(valid_target, forest_upsampled_valid_predictions) > best_f1:
            best_accuracy = accuracy_score(valid_target, forest_upsampled_valid_predictions)
            best_f1 = f1_score(valid_target, forest_upsampled_valid_predictions)
            best_auc_roc = roc_auc_score(valid_target, forest_upsampled_prob_one)
            best_depth = depth
            best_est = est
            best_model_forest_upsampled = model_forest_upsampled
            
print('Best n_estimators = ', best_est, 'best depth =', best_depth)
print('Accuracy =', best_accuracy)
print('F1 =', best_f1)
print('auc_roc =', best_auc_roc)
print('')

Best n_estimators =  15 best depth = 12
Accuracy = 0.8205
F1 = 0.5943502824858756
auc_roc = 0.8340644188101816



**Логистическая регрессия**

In [20]:
model_logreg_upsampled = LogisticRegression(random_state=123, solver='liblinear')
model_logreg_upsampled.fit(train_features_upsampled, train_target_upsampled)
logreg_upsampled_valid_predictions = model_logreg_upsampled.predict(valid_features)
logreg_upsampled_prob = model_logreg_upsampled.predict_proba(valid_features)
logreg_upsampled_prob_one = logreg_upsampled_prob[:, 1]

print('Accuracy =', accuracy_score(valid_target, logreg_upsampled_valid_predictions))
print('F1 =', f1_score(valid_target, logreg_upsampled_valid_predictions))
print('auc_roc =', roc_auc_score(valid_target, logreg_upsampled_prob_one))

Accuracy = 0.6465
F1 = 0.4835646457268079
auc_roc = 0.7692376505935828


### Downsampling

In [21]:
def downsample(features, target, fraction):
    
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state = 123)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state = 123)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=123)
    
    return features_downsampled, target_downsampled

train_features_downsampled, train_target_downsampled = downsample(train_features, train_target, 0.2)

print(train_features.shape)
print(train_features_downsampled.shape)
print(train_target_downsampled.shape)

(6000, 11)
(2178, 11)
(2178,)


**Дерево решений**

In [22]:
best_accuracy = 0
best_f1 = 0
best_auc_roc = 0
best_depth = 0
best_model_tree_downsampled = None

for depth in range(1,16):      
    model_tree_downsampled = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model_tree_downsampled.fit(train_features_downsampled, train_target_downsampled)
    tree_downsampled_valid_predictions = model_tree_downsampled.predict(valid_features)
    tree_downsampled_prob = model_tree_downsampled.predict_proba(valid_features)
    tree_downsampled_prob_one = tree_downsampled_prob[:, 1]
    
    if f1_score(valid_target, tree_downsampled_valid_predictions) > best_f1:
        best_accuracy = accuracy_score(valid_target, tree_downsampled_valid_predictions)
        best_f1 = f1_score(valid_target, tree_downsampled_valid_predictions)
        best_auc_roc = roc_auc_score(valid_target, tree_downsampled_prob_one)
        best_depth = depth
        best_model_tree_downsampled = model_tree_downsampled
        
print('Best depth =', best_depth)
print('Accuracy =', best_accuracy)
print('F1 =', best_f1)
print('auc_roc =', best_auc_roc)
print('')

Best depth = 5
Accuracy = 0.727
F1 = 0.5531914893617021
auc_roc = 0.8240251036861206



**Случайный лес**

In [23]:
best_accuracy = 0
best_f1 = 0
best_auc_roc = 0
best_depth = 0
best_est = 0
best_model_forest_downsampled = None

for est in range(1, 16):
    for depth in range(1, 16):
        model_forest_downsampled = RandomForestClassifier(n_estimators = est, random_state=123, max_depth=depth)
        model_forest_downsampled.fit(train_features_downsampled, train_target_downsampled)
        forest_downsampled_valid_predictions = model_forest_downsampled.predict(valid_features)
        forest_downsampled_prob = model_forest_downsampled.predict_proba(valid_features)
        forest_downsampled_prob_one = forest_downsampled_prob[:, 1]
    
        if f1_score(valid_target, forest_downsampled_valid_predictions) > best_f1:
            best_accuracy = accuracy_score(valid_target, forest_downsampled_valid_predictions)
            best_f1 = f1_score(valid_target, forest_downsampled_valid_predictions)
            best_auc_roc = roc_auc_score(valid_target, forest_downsampled_prob_one)
            best_depth = depth
            best_est = est
            best_model_forest_downsampled = model_forest_downsampled
            
print('Best n_estimators = ', best_est, 'best depth =', best_depth)
print('Accuracy =', best_accuracy)
print('F1 =', best_f1)
print('auc_roc =', best_auc_roc)
print('')

Best n_estimators =  13 best depth = 7
Accuracy = 0.728
F1 = 0.5555555555555556
auc_roc = 0.8426114866792833



**Логистическая регрессия**

In [24]:
model_logreg_downsampled = LogisticRegression(random_state=123, solver='liblinear')
model_logreg_downsampled.fit(train_features_downsampled, train_target_downsampled)
logreg_downsampled_valid_predictions = model_logreg_downsampled.predict(valid_features)
logreg_downsampled_prob = model_logreg_downsampled.predict_proba(valid_features)
logreg_downsampled_prob_one = logreg_downsampled_prob[:, 1]

print('Accuracy =', accuracy_score(valid_target, logreg_downsampled_valid_predictions))
print('F1 =', f1_score(valid_target, logreg_downsampled_valid_predictions))
print('auc_roc =', roc_auc_score(valid_target, logreg_downsampled_prob_one))

Accuracy = 0.6375
F1 = 0.4772891131939438
auc_roc = 0.7678927000960899


### Вывод

Было применено два метода устранения дисбаланса: upsampling и downsampling. Результаты обучения моделей в обоих случаях похожи, однако, при обучении модели случайного леса, получены лучшие метрики на данных, обработанных методом upsample.

Самыми лучшими метриками обладает модель случайного леса, обученная на данных, обработанных методом upsample.

## Тестирование модели

Проверим лучшую из полученных моделей на тестовой выборке

In [25]:
test_predictions = best_model_forest_upsampled.predict(test_features)
test_prob = best_model_forest_downsampled.predict_proba(test_features)
test_prob_one = test_prob[:, 1]

print('accuracy =', accuracy_score(test_target, test_predictions))
print('f1 =', f1_score(test_target, test_predictions))
print('auc_roc =', roc_auc_score(test_target, test_prob_one))

accuracy = 0.827
f1 = 0.6094808126410836
auc_roc = 0.8477961783046529


## Вывод

В работе было необходимо спрогнозировать уход клиента из банка.

Данные были предварительно подготовлены:
1. Закодированы категориальные признаки
2. Разбиты на обучающую, валидационную и тестовую выборки
3. Признаки масштабированы

Были обучены три модели: дерево решений, случайный лес, логистическая регрессия.

Результаты предсказаний, полученные на данных без учёта дисбаланса классов, оказались неудовлетварительными. 

Дисбаланс классов был устранен двумя методами: upsampling и downsampling. В результате, наилучшая модель получилась на данных, обработанных методом upsampling.

Удалось достичь значения метрики f1 = 0.59 и подтвердить данное значение на тестовой выборке. При этом, метрика accuracy превышает аналогичную для константной модели всего на 0.04. А значение метрики AUC-ROC равно 0.858, что далеко до идеала, но значительно превышает значение случайной модели.